*This notebook covers how to run and load a basic `DESeq2` DEG result as a `GSForge.GeneSet`.*

---

#### Notebook Preparation

***Declare used paths***

In [1]:
# OS-independent path management.
from os import fspath, environ
from pathlib import Path

In [2]:
OSF_PATH = Path(environ.get("GSFORGE_DEMO_DATA", default="~/GSForge_demo_data")).expanduser()
AGEM_PATH = OSF_PATH.joinpath("osfstorage", "rice.nc")
DEG_COLL_PATH = OSF_PATH.joinpath("osfstorage", "DEG_gene_sets")
assert AGEM_PATH.exists()

***Import Python packages***

In [3]:
import GSForge as gsf

***R integration setup***

In [5]:
import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
%load_ext rpy2.ipython
pandas2ri.activate()
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

/home/tyler/anaconda3/envs/py38/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


***Import R Packages***

In [8]:
%%R
library("DESeq2")

***Loading an AnnotatedGEM***

In [9]:
agem = gsf.AnnotatedGEM(AGEM_PATH)
agem

<GSForge.AnnotatedGEM>
Name: Rice
Selected GEM Variable: 'counts'
    Gene   55986
    Sample 475

### Prepare input data for DESeq2

This requires us to drop genes that have counts of zero.

In [10]:
dropped_counts, labels = gsf.get_data(agem, 
                                      count_mask="dropped",
                                      annotation_variables=["Treatment"])

These counts were made with Kallisto, so we must round them for use in `DEseq2`.

In [11]:
dropped_counts

<xarray.DataArray 'counts' (Sample: 475, Gene: 10593)>
array([[200., 102., 118., ...,  24., 342., 637.],
       [ 41.,  55.,  29., ...,  10., 152., 186.],
       [197., 104.,  78., ...,  21., 336., 545.],
       ...,
       [158., 113., 225., ...,   9., 424., 411.],
       [151.,  98., 151., ...,  15., 274., 311.],
       [128., 110., 105., ...,  19., 335., 666.]])
Coordinates:
  * Gene     (Gene) object 'LOC_Os01g55490' ... 'LOC_Os03g20020'
  * Sample   (Sample) object 'SRX1423934' 'SRX1423935' ... 'SRX1424408'

***Round counts to intergers***

In [12]:
ri_dropped_counts = gsf.utils.R_interface.Py_counts_to_R(dropped_counts)
ri_dropped_counts = ri_dropped_counts.round()

ri_labels = labels.to_dataframe()

In [13]:
ri_dropped_counts.head(2)

Sample,SRX1423934,SRX1423935,SRX1423936,SRX1423937,SRX1423938,SRX1423939,SRX1423940,SRX1423941,SRX1423942,SRX1423943,...,SRX1424399,SRX1424400,SRX1424401,SRX1424402,SRX1424403,SRX1424404,SRX1424405,SRX1424406,SRX1424407,SRX1424408
Gene,,,,,,,,,,,,,,,,,,,,,
ChrSy.fgenesh.gene.37,547.0,173.0,626.0,161.0,404.0,728.0,439.0,1011.0,532.0,705.0,...,543.0,695.0,601.0,501.0,463.0,611.0,336.0,326.0,387.0,596.0
ChrSy.fgenesh.gene.49,136.0,34.0,149.0,30.0,134.0,275.0,134.0,301.0,147.0,134.0,...,179.0,116.0,125.0,191.0,261.0,154.0,220.0,168.0,152.0,123.0


In [14]:
ri_labels.head(2)

,Treatment
Sample,
SRX1423934,CONTROL
SRX1423935,CONTROL


### `DESeq2` Runs

In [17]:
%%R -i ri_dropped_counts -i ri_labels -o deseq_df

dds <- DESeqDataSetFromMatrix(countData = ri_dropped_counts,
                              colData = ri_labels,
                              design= ~ Treatment)
dds <- DESeq(dds)
deseq_results <- results(dds)
deseq_df = data.frame(deseq_results)

In [18]:
deseq_df.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ChrSy.fgenesh.gene.37,568.392376,-0.226739,0.091348,-2.482149,1.305927e-02,2.269306e-02
ChrSy.fgenesh.gene.49,114.847307,0.129674,0.048751,2.659933,7.815617e-03,1.425953e-02
ChrSy.fgenesh.gene.74,89.416241,-0.265554,0.051419,-5.164514,2.410650e-07,9.272336e-07
ChrSy.fgenesh.gene.86,979.544110,0.108172,0.094211,1.148190,2.508903e-01,3.177904e-01
ChrUn.fgenesh.gene.94,95.108651,-0.735490,0.079190,-9.287674,1.576965e-20,2.956600e-19


In [19]:
deseq2_treatment = gsf.GeneSet(deseq_df, 
                               name="deseq2_treatment", 
                               attrs={"DESeq2_formula": "~ Treatment"})
deseq2_treatment

<GSForge.GeneSet>
Name: deseq2_treatment
    Supported Genes:  10593, 100.00% of 10593

### Define Helper Functions

Some functions to help assign support to this `GeneSet`.

In [20]:
def pvalue_filter(deseq_result_df, cutoff=0.05):
    """Returns a array of genes which have p-values above the specified cutoff."""
    return deseq_result_df[deseq_result_df["padj"] < cutoff].index

def top_n_abs(dataframe, n=10, col="log2FoldChange", padj_cuttoff=0.05):
    """Returns the top n most (absolutely) differentially expressed genes from a deseq2 result.
    This also filters by p-values."""
    filtered_df = dataframe[dataframe["padj"] < padj_cuttof]
    filtered_df = filtered_df.reindex(filtered_df["log2FoldChange"].abs().sort_values().index)
    return filtered_df.tail(n).index

In [21]:
cutoff = 0.05
gene_count = len(pvalue_filter(deseq_df, cutoff=cutoff))

print(f"{gene_count} genes below P-value threshold of: {cutoff}")

6659 genes below P-value threshold of: 0.05


In [22]:
deseq2_treatment.set_support_by_genes(pvalue_filter(deseq_df, cutoff=cutoff))
deseq2_treatment

<GSForge.GeneSet>
Name: deseq2_treatment
    Supported Genes:  6659, 62.86% of 10593

In [23]:
deseq2_treatment.save_as_netcdf(DEG_COLL_PATH)

'/home/tyler/GSForge_demo_data/osfstorage/DEG_gene_sets/deseq2_treatment.nc'

---